# Regressão logística

### O que é?
Podemos resumir regressão logística em uma equação que busca classificar informações a partir do o resultado de uma série de dados. A classificação é binária, podendo sendo "não" (0) ou "sim" (1). O resultado é encontrado em porcentagem, no qual o valor varia de 0 a 1. O resultado é encontrado em porcentagem, aonde o valor varia de 0 a 1.

>A regressão logística estima a probabilidade de ocorrência de um evento, como voto ou não voto, com base em um determinado conjunto de dados de variáveis independentes.
>
> \- O que é regressão logística? | IBM. Disponível em: <<https://www.ibm.com/br-pt/topics/logistic-regression>>.


A equação a seguir descreve a regressão logística:

<img src="../assets/image.png" />


A partir disso podemos encontrar os valores e classificá-los.
Para testar os valores encontrados, podemos usar um método chamado de "teste de Hosmer-Lemeshow".
> Depois que o modelo tiver sido calculado, a melhor prática é avaliar o quanto o modelo prevê a variável dependente, o que é chamado de grau de adequação. O teste de Hosmer-Lemeshow é um método popular para avaliar a adequação do modelo.
>
> \- O que é regressão logística? | IBM. Disponível em: <<https://www.ibm.com/br-pt/topics/logistic-regression>>.

### Primeiramente vamos instalar as dependencias
Temos que instalar as seguintes dependências

[Pandas - https://pandas.pydata.org/](https://pandas.pydata.org/) 

[Scikit-learn - https://scikit-learn.org/stable/](https://scikit-learn.org/stable/)

[Seaborn - https://seaborn.pydata.org/](https://seaborn.pydata.org/)

[Matplotlib - https://matplotlib.org/](https://matplotlib.org/)

In [ ]:
pip install -U scikit-learn pandas seaborn matplotlib unicode --break-system-packages

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model as lm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from datetime import datetime
import unicodedata
import plotly.express as px

In [ ]:
# Função para remover acentos
def remove_acentos(text):
  return ''.join(
    char for char in unicodedata.normalize('NFD', text)
    if unicodedata.category(char) != 'Mn'
  )

def normalize_data(data):
  min_data = np.min(data)
  max_data = np.max(data)

  return (data - min_data) / (max_data - min_data)

def dect_Outliers(dados):
  q1 = np.percentile(dados,25)
  q3 = np.percentile(dados,75)

  iiq = q3 - q1   

  min = q1 - 1.5 * iiq
  max = q3 + 1.5 * iiq

  outliers = [x for x in dados if x < min or x > max]

  return outliers

In [ ]:
matches = pd.read_csv('../../assets/matches.csv', sep=';')
teams = pd.read_csv('../../assets/teams.csv', sep=',')
matches_2 = pd.read_csv('../../assets/BRA.csv', sep=',')

matches_2['formated_date'] = pd.to_datetime(matches_2['Date'], format='%d/%m/%Y')

matches_2.rename(columns={
  'Home': 'home_team_name', 
  'Away': 'away_team_name',
  'HG': 'home_team_goal_count',
  'AG': 'away_team_goal_count'}, inplace=True)

# Captura apenas os dados do Brasileirão 2024
matches_2 = matches_2.query('formated_date >= "2024-04-01"')

# Aplicar a função a uma coluna específica (ex: 'nome')
matches['home_team_name'] = matches['home_team_name'].apply(remove_acentos)
matches['away_team_name'] = matches['away_team_name'].apply(remove_acentos)
matches_2['home_team_name'] = matches_2['home_team_name'].apply(remove_acentos)
matches_2['away_team_name'] = matches_2['away_team_name'].apply(remove_acentos)
teams['common_name'] = teams['common_name'].apply(remove_acentos)

teams = teams.drop(columns=['team_name', 'country', 'season'])
matches = matches.drop(columns=['timestamp'])

### Organização e pré-processamento de dados

Aqui iremos organizar os dados em uma estrutura que nosso modelo reconheça

A ideia é:

- Encontrar os times que jogam entre si
- Capturar os dados do time A
- Capturar os dados do time B
- Capturar o status da partida
- Adicionar tudo em uma linha

In [ ]:
teams_results = []

new_team_names = [
  'Vitória',
  'Flamengo RJ',
  'Cruzeiro',
  'Botafogo RJ',
  'Grêmio',
  'Fluminense',
  'Sao Paulo',
  'Palmeiras',
  'Atletico-MG',
  'Athletico-PR',
  'Corinthians',
  'Vasco',
  'Bahia',
  'Atlético GO',
  'Internacional',
  'Bragantino',
  'Criciuma',
  'Juventude',
  'Cuiaba',
  'Fortaleza'
]

resolved_team_names = teams
resolved_team_names['common_name'] = new_team_names
resolved_team_names['common_name'] = resolved_team_names['common_name'].apply(remove_acentos)
resolved_team_names.head()

teams_confrontations = []

for index, curr_match in matches_2.iterrows():

  result = curr_match.home_team_goal_count - curr_match.away_team_goal_count
  confrontation = {"winner": 0, "color": 'blue'} # 0 é empate
  if result > 0:
    confrontation['winner'] = 1 # 1 é o time da casa
    confrontation['color'] = 'verde' # 1 é o time da casa
  elif result < 0:
    confrontation['winner'] = 2 # 2 é o time visitante
    confrontation['color'] = 'vermelho' # 1 é o time da casa

  away_data = resolved_team_names.query('common_name == @curr_match.away_team_name')
  home_data = resolved_team_names.query('common_name == @curr_match.home_team_name')

  away_data = away_data.rename(columns=lambda col: f'{col}_a')
  home_data = home_data.rename(columns=lambda col: f'{col}_h')
  
  # Concatena os dataframes
  confrontation.update(away_data.iloc[0].to_dict())
  confrontation.update(home_data.iloc[0].to_dict())
  
  teams_confrontations.append(confrontation)

teams_confrontations = pd.DataFrame(teams_confrontations)
teams_confrontations.to_csv('./teams_confrontations.csv', index=False)



### Entendendo os dados
O gráfico a baixo visa entender a diferença na quantidade de dados entre empates e vitórias
- 0 = Empate
- 1 = Vitória casa
- 2 = Vitória visitante

No caso do gráfico a baixo, podemos identificar que há muito mais resultados que deram vitória da casa.
Isso atrapalha o modelo pois ele fica enviesado.

Para evitar isso, precisamos realizar a limpeza dos dados, igualando o número de vitórias.

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(20, 10))

teams_confrontations['winner'].value_counts().plot(kind='bar', ax=ax[0])
teams_confrontations['winner'].value_counts().plot(kind='pie', ax=ax[1])

plt.show()

In [ ]:
home_wins = teams_confrontations.query('winner == 1')
cleaned_teams_confrontations = teams_confrontations.query('winner != 1')

cleaned_teams_confrontations = pd.concat([cleaned_teams_confrontations, home_wins.sample(n=57, random_state=42)])
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(20, 10))

cleaned_teams_confrontations['winner'].value_counts().plot(kind='bar', ax=ax[0])
cleaned_teams_confrontations['winner'].value_counts().plot(kind='pie', ax=ax[1])

plt.show()

### Treinamento e testagem
Inicia o treinamento do modelo com os dados gerados

Além disso gerei uma tabela que demonstra os pesos de todas as colunas

In [ ]:
# pega as colunas com coeficientes para 0, 1 e 2 que são menores que 0
X = teams_confrontations.drop(columns=['winner', 'common_name_a', 'common_name_h', 'color'], axis=1)
y = teams_confrontations['winner']

feature_names = X.columns

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

model = lm.LogisticRegression(solver='lbfgs', max_iter=5000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Avaliar a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy * 100:.2f}%")

# Adiciona os valores de x as precições
X_test['winner'] = y_pred
X_test = pd.DataFrame(X_test)
X_test.to_csv('./predictions.csv', index=False)

pd.DataFrame(model.coef_, columns=feature_names).T.to_csv('./coeficientes.csv', index=True)

### Matriz de confusão
Aqui conseguimos ver a matriz de confusão, que mostra os valores corretos contra os valores previstos pela IA

In [ ]:
cm = confusion_matrix(y_test, y_pred)

# Plotar a matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()